In [1]:
import numpy as np
import pandas as pd
import cv2

In [2]:
# Global variables

video_path = "Video/Easy/T002_ActionsShorter_mini_11552_12166_Dance-Sing.mp4"
retina_csv_path = "RetinaFaceDetections/Easy/T002_ActionsShorter_mini_11552_12166_Dance-Sing.csv"

In [3]:
def returnRetinaDf(retina_csv_path):
    """
    Read retina detections csv file into dataframe
    
    Input: file path to csv
    """
    retina_df = pd.read_csv(retina_csv_path)
    retina_df = retina_df[["x", "y", "w", "h"]]
    return retina_df

In [4]:
retina_df = returnRetinaDf(retina_csv_path)
retina_df.head(5)

,x,y,w,h
0,164.107300,66.408592,135.621979,180.596283
1,164.009872,65.886246,135.752701,181.113007
2,164.514114,66.950378,135.537827,180.016693
3,164.875748,67.175591,135.459274,179.871582
4,164.988770,67.111977,135.418457,180.024536


In [5]:
# example submission 
def pixelate(img, size):
    height, width = img.shape[:2]
    w, h = (size, size)
    temp = cv2.resize(img, (w, h), interpolation=cv2.INTER_LINEAR)
    output = cv2.resize(temp, (width, height), interpolation=cv2.INTER_NEAREST)
    return output

In [6]:
# example submission
def sharpen_avg(img):
    # sharpening filter
    img = cv2.addWeighted(img, 4, cv2.blur(img, (30, 30)), -4, 128)
    # averaging kernel; apply vertical kernel, then horizontal kernel
    img = cv2.blur(img, (20, 10))
    img = cv2.blur(img, (10, 20))
    return img

In [7]:
# example submissions
def median_avg(img):
    # median avg filter
    return cv2.medianBlur(img, 27)


In [8]:
def returnDataMatrix(video_path):
    
    """
    Takes in video path and returns data matrix
    """
    
    cap = cv2.VideoCapture(video_path)
    frame_counter = 0

    data_matrix = []
    
    ##### Remove this ##################################################################
    pixelated_matrix = []
    sharpen_blur_matrix = []
    median_matrix = []
    ###################################################################################

    while cap.isOpened():

        ret,frame = cap.read()
        if frame is None:
            break


        x = int(retina_df["x"][frame_counter])
        y = int(retina_df["y"][frame_counter])
        l = max(int(retina_df["w"][frame_counter]), int(retina_df["h"][frame_counter]))

        frame = frame[y:y + l, x: x + l]
        frame = cv2.resize(frame, (224, 224), interpolation = cv2.INTER_AREA)
        
        ##### Remove this #############################################################################
        pixelated_frame = pixelate(frame, 65)
        sharpen_blur_frame = sharpen_avg(frame)
        median_frame = median_avg(frame)
        #cv2.imwrite("pixelated.jpg", pixelated_frame)
        #cv2.imwrite("sharpen_blur.jpg", sharpen_blur_frame)
        #cv2.imwrite("median.jpg", median_frame)
        ##############################################################################################
        
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = [j for i in frame for j in i]
        
        ##### Remove this #############################################################################
        pixelated_frame = cv2.cvtColor(pixelated_frame, cv2.COLOR_BGR2GRAY)
        #cv2.imwrite("bw_pixelated.jpg", pixelated_frame)
        pixelated_frame = [j for i in pixelated_frame for j in i]
        
        sharpen_blur_frame = cv2.cvtColor(sharpen_blur_frame, cv2.COLOR_BGR2GRAY)
        #cv2.imwrite("bw_sharpen_blur.jpg", sharpen_blur_frame)
        sharpen_blur_frame = [j for i in sharpen_blur_frame for j in i]
        
        median_frame = cv2.cvtColor(median_frame, cv2.COLOR_BGR2GRAY)
        #cv2.imwrite("bw_median.jpg", median_frame)
        median_frame = [j for i in median_frame for j in i]
        ##############################################################################################

        data_matrix.append(frame)
        
        ##### Remove this #############################################################################
        pixelated_matrix.append(pixelated_frame)
        sharpen_blur_matrix.append(sharpen_blur_frame)
        median_matrix.append(median_frame)
        ##############################################################################################

        frame_counter += 1


    cap.release()

    data_matrix = np.array(data_matrix)
    #### Remove this ###########################
    pixelated_matrix = np.array(pixelated_matrix)
    sharpen_blur_matrix = np.array(sharpen_blur_matrix)
    median_matrix = np.array(median_matrix)
    #######################################################
    return data_matrix, pixelated_matrix, sharpen_blur_matrix, median_matrix # change to: return data_matrix

In [9]:
data_matrix, pixelated_matrix, sharpen_blur_matrix, median_matrix = returnDataMatrix(video_path)

In [10]:
data_matrix.shape

(660, 50176)

In [11]:
pixelated_matrix.shape

(660, 50176)

In [12]:
sharpen_blur_matrix.shape

(660, 50176)

In [13]:
median_matrix.shape

(660, 50176)

In [23]:
from sklearn.decomposition import PCA
from sklearn.decomposition import SparsePCA
from sklearn.decomposition import TruncatedSVD

In [34]:
def pca_error(matrix, de_identified_matrix):
    
    """
    matrix: corresponds to original video
    de_identified_matrix: correponds to de-identified video
    """
    
    pca = PCA(n_components=min(matrix.shape[0], matrix.shape[1]), svd_solver='full')
    pca.fit(matrix)
    
    dim_data_matrix = pca.transform(matrix)
    dim_de_identified_matrix = pca.transform(de_identified_matrix)
    
    return np.linalg.norm(dim_de_identified_matrix - dim_data_matrix)  

In [35]:
print("pixelated error: " + str(pca_error(data_matrix, pixelated_matrix)))
print("sharpen blur error: " + str(pca_error(data_matrix, sharpen_blur_matrix)))
print("median error: " + str(pca_error(data_matrix, median_matrix)))

pixelated error: 35153.80663509222
sharpen blur error: 211614.6274278321
median error: 67324.27743337234


In [32]:
def sparse_pca_error(matrix, matrix1, matrix2, matrix3):
    
    """
    matrix: corresponds to original video
    matrix1, matrix2, matrix3, .... : correspond to de-identified videos
    """
    
    svd = TruncatedSVD(n_components = int(min(matrix.shape[0], matrix.shape[1]) * 0.75))
    svd.fit(matrix)
    matrix = svd.transform(matrix)
    print(matrix.shape)
    
    sparse_pca = SparsePCA(n_components = 5, alpha=1.5)
    sparse_pca.fit(matrix)
    dim_data_matrix = sparse_pca.transform(matrix)
    
    def compute_error(de_identified_matrix):
        
        svd = TruncatedSVD(n_components = int(min(de_identified_matrix.shape[0], de_identified_matrix.shape[1]) * 0.75))
        svd.fit(de_identified_matrix)
        de_identified_matrix = svd.transform(de_identified_matrix)
        
        dim_de_identified_matrix = sparse_pca.transform(de_identified_matrix)

        return np.linalg.norm(dim_de_identified_matrix-dim_data_matrix)
    
    return compute_error(matrix1), compute_error(matrix2), compute_error(matrix3)

In [33]:
pixelated_error, sharpen_blur_error, median_error = sparse_pca_error(data_matrix, pixelated_matrix, sharpen_blur_matrix, median_matrix)
print("pixelated error: " + str(pixelated_error))
print("sharpen blur error: " + str(sharpen_blur_error))
print("median error: " + str(median_error))

(660, 495)
pixelated error: 2851.4223726982395
sharpen blur error: 155355.21643295544
median error: 26712.416983700863
